# Three-bus power flow example

Example 5.4 given at page 89 of Ned Mohan's Book. Based on network shown at page 80 and gradually described in the other examples.

Buses: 

 - Bus 1 is the slack, with voltage at 1 pu
 - Bus 2 is a PV bus, with voltage at 1.05 pu and drawing 2 pu of active power
 - Bus 3 is a PQ bus, consumed P = 5 pu and Q = 1 pu.

Lines: 

 - Shunt susceptances are ignored (4.5e-6 S/km)
 - X = 0.376 Ohm/km (at 6h Hz)
 - R = 0.037 Ohm/km
 - 1 to 2: 150 km
 - 2 to 3: 150 km
 - 3 to 1: 200 km
 
 
Voltage base (3-phase): 345 kV

Power base (3-phase): 100 MVA

TODO: 

 - base values
 - line params

In [5]:
import pandapower as pp
#create empty net
net = pp.create_empty_network() 

# Per unit (3-phase values)
Pbase = 100  # MVA
Vbase = 345 # kV

#create buses
b1 = pp.create_bus(net, vn_kv=Vbase, name="Bus 1")
b2 = pp.create_bus(net, vn_kv=Vbase, name="Bus 2")
b3 = pp.create_bus(net, vn_kv=Vbase, name="Bus 3")

#create bus elements
pp.create_ext_grid(net, bus=b1, vm_pu=1.00, name="Grid Connection")
pp.create_load(net, bus=b3, p_mw=5*Pbase, q_mvar=1*Pbase, name="Load")
pp.create_gen(net, bus=b2, p_mw=2*Pbase, vm_pu=1.05, name="PV")

#create branch elements. Here I use impedances since we do not have accurate info about line types.
Zbase = Vbase**2 / Pbase
X_km = 0.376
R_km = 0.037
l12_km = 150
l23_km = 150
l31_km = 200

pp.create_impedance(net, from_bus=b1, to_bus=b2, rft_pu=l12_km*R_km/Zbase, rtf_pu=l12_km*R_km/Zbase, xft_pu=l12_km*X_km/Zbase, xtf_pu=l12_km*X_km/Zbase, sn_mva=Pbase, in_service=True)
pp.create_impedance(net, from_bus=b2, to_bus=b3, rft_pu=l23_km*R_km/Zbase, rtf_pu=l23_km*R_km/Zbase, xft_pu=l23_km*X_km/Zbase, xtf_pu=l23_km*X_km/Zbase, sn_mva=Pbase, in_service=True)
pp.create_impedance(net, from_bus=b3, to_bus=b1, rft_pu=l31_km*R_km/Zbase, rtf_pu=l31_km*R_km/Zbase, xft_pu=l31_km*X_km/Zbase, xtf_pu=l31_km*X_km/Zbase, sn_mva=Pbase, in_service=True)

2

## Run the power flow and analyze the results

In [6]:
pp.runpp(net)

In [7]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-308.377053,81.610978
1,1.050000,-2.066481,-200.000000,-266.739953
2,0.978114,-8.787327,500.000000,100.000000


In [4]:
net.res_impedance

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka
0,68.993240,-110.866762,-68.198148,118.946619,0.795092,8.079857,0.218525,0.218525
1,268.198148,147.793334,-264.232131,-107.490029,3.966017,40.303305,0.488057,0.488057
2,-235.767869,7.490029,239.383813,29.255784,3.615944,36.745813,0.403584,0.403584


BCR's comment, results are coherent with solution from the book (at first attempt!)

## Add a transformer between buses 1 and 3 to regulate voltage 

See 6.18 and 6.19 in the book, and page 22 in the lab manual